<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        This is an extension notebook for <a href="https://www.thelmbook.com" target="_blank" rel="noopener">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a href="https://github.com/aburkov/theLMbook" target="_blank" rel="noopener">https://github.com/aburkov/theLMbook</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>

# 从零开始编写GRPO：使用Qwen2.5-1.5B-Instruct实现分布式指南

> 如果您购买了[这本书](https://thelmbook.com/)，您可以通过发送邮件至[author@thelmbook.com](mailto:author@thelmbook.com)向[Lambda](https://lambdalabs.com/)申请$150的免费云GPU积分。使用这些云积分，您可以启动一个8xA100实例，并按照本教程的所有步骤进行操作，包括分布式训练。

在本教程中，我们将演示如何使用GRPO（Group Relative Policy Optimization）方法构建一个分布式强化学习（RL）管道，以微调一个用于数学、逻辑和编码任务的语言模型。这些任务存在唯一的正确答案，可以通过简单的字符串比较轻松地验证其与真实答案的一致性。

GRPO由DeepSeek发明，并用于微调DeepSeek R1和R1-Zero模型，使其通过生成思维链（CoT）在数学和逻辑任务中表现出色。您可以在[这篇文章](https://thelmbook.com/articles/#!./DeepSeek-R1.md)中找到关于R1和R1-Zero训练的详细概述。

本教程的目标是将通用语言模型**Qwen2.5-1.5B-Instruct**转变为数学问题解决器。我们将从零开始编写GRPO代码，然后将其与多个流行的库和工具集成，以实现分布式训练管道，包括：

- **PyTorch:** 用于张量操作和分布式训练。
- **Hugging Face Transformers:** 用于加载预训练的语言模型和分词器。
- **FlashAttention2:** 用于优化注意力机制，帮助减少内存使用并提高训练速度。
- **Weights & Biases (wandb):** 用于实验跟踪、可视化和模型版本控制。

本教程分为几个部分。我们从基本设置和导入开始，然后转到数据格式化和答案提取、数据集准备、评估函数、奖励函数、训练设置和执行，最后加载和测试模型。在此过程中，我们从零开始实现GRPO算法。

## 第一部分：基本设置和导入

在第一部分中，我们安装并导入所有必要的模块。我们还通过配置随机种子以确保可重复性，并初始化实验跟踪所需的环境变量。此外，我们安装并导入提供优化Transformer注意力机制（FlashAttention2）和报告（Weights and Biases）的库。


In [1]:
# !pip install tf-keras # for some reason, Hugging Face cannot work without it
# !pip install flash-attn # FlashAttention2
# !pip install wandb # Weights and Biases
# !pip install 'accelerate>=0.26.0'
# !pip install transformers # Hugging Face Transformers API
# !pip install datasets # Hugging Face Datasets API

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"
os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


import warnings
warnings.filterwarnings("ignore")

from tqdm.auto import tqdm

# Import necessary libraries
# Basic Python libraries for various operations
import random
import copy
import re
import os
import numpy as np
import wandb

# PyTorch and related libraries for deep learning
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import deepspeed

# Hugging Face libraries for transformer models
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# 使用镜像站下载
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'


def set_random_seed(seed: int = 42):
    """
    Set the random seed for reproducibility across Python, NumPy, and PyTorch.

    Args:
        seed (int): The seed value to use for random number generation.

    Returns:
        None

    Explanation:
        1. Sets seed for Python's built-in random module for basic random operations.
        2. Sets seed for NumPy, ensuring consistent random number generation in array operations.
        3. Sets seed for PyTorch CPU operations.
        4. If CUDA is available, sets seed for all GPU devices.
        5. Configures cuDNN to ensure deterministic behavior:
           - Sets deterministic flag to True, ensuring reproducible results.
           - Disables benchmarking to prevent algorithm selection based on hardware.

    Note:
        Setting deterministic behavior may impact performance but ensures consistent results
        across multiple runs, which is crucial for debugging and research.
    """
    # Set the seed for Python's built-in random module
    random.seed(seed)
    # Set the seed for NumPy
    np.random.seed(seed)
    # Set the seed for PyTorch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    # Ensure deterministic behavior in cuDNN (may impact performance)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Call the function to set random seed for reproducibility
set_random_seed(42)

from torch.utils.tensorboard import SummaryWriter
# 不使用这一部分, 改用tensorboard
# Set environment variables for Weights & Biases (wandb) logging
# os.environ["WANDB_API_KEY"] = ""
# os.environ["WANDB_PROJECT"] = "GRPO-Qwen-1.5-Instruct-Multi-GPU"

[2025-03-08 11:21:14,625] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/yeqi3/anaconda3/envs/cyr/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/yeqi3/anaconda3/envs/cyr/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/yeqi3/anaconda3/envs/cyr/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/yeqi3/anaconda3/envs/cyr/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/yeqi3/anaconda3/envs/cyr/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/yeqi3/anaconda3/envs/cyr/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/home/yeqi3/anaconda3/envs/cyr/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.

上述代码执行以下任务：
- **设置随机种子：** `set_random_seed` 函数通过为Python的随机模块、NumPy和PyTorch设置种子来确保可重复性。它还配置了PyTorch的cuDNN后端以实现确定性行为。
- **环境变量配置：** 我们设置了 `WANDB_API_KEY` 和 `WANDB_PROJECT` 环境变量，以启用Weights & Biases的实验跟踪功能。
- **导入必要的包：** 脚本导入了管道所需的所有必要模块，包括：
以下是对每个导入的详细说明：
- **random：** 用于数据集洗牌和随机操作。
- **copy：** 提供深度复制对象的功能。
- **re：** 提供正则表达式支持以进行文本处理。
- **numpy (np)：** 支持数值操作和数组处理。
- **torch：** 提供GPU加速的张量操作和深度学习原语。
- **torch.nn：** 包含神经网络模块和操作。
- **pad_sequence：** 处理可变长度序列的填充以进行批处理。
- **AutoTokenizer** 和 **AutoModelForCausalLM：** 加载预训练的语言模型及其分词器。
- **load_dataset：** 从Hugging Face的datasets库加载数据集。

## 第二部分：数据格式化和答案提取
在本节中，我们定义了数据的格式化方式以及如何从模型输出和数据集中提取答案部分。为了确保


In [2]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
And must have '####' before the final answer number.
"""

# 从模型输出中获取<answer>和</answer>之间的部分
def extract_answer_from_model_output(text):
   """
   Extracts the value from the last <answer> tag in the text.

   Args:
       text (str): The model-generated text containing XML-style <answer> tags.

   Returns:
       str or None: The content inside the <answer> tags, or None if no valid answer is found.

   Explanation:
       1. Splits the text on the <answer> tag to isolate content after the tag.
       2. Checks if at least one <answer> tag exists in the text.
       3. For the last <answer> segment:
          - Verifies it contains a closing </answer> tag.
          - Extracts only the content between the tags.
       4. Returns None if the answer is empty (just "...") or if tags are missing.
   """
   # Split on <answer> and take everything after the last occurrence
   parts = text.split("<answer>")
   # 如果没有找到<answer>，那就是没有进行切割，长度只有1
   if len(parts) < 2:  
       return None
   last_part = parts[-1]  # 只取最后一个<answer>部分

   # 结尾没有</answer>
   if "</answer>" not in last_part:
       return None
   answer = last_part.split("</answer>")[0].strip()
   return None if answer == "..." else answer

# 从数据集中提取答案
def extract_answer_from_dataset(text):
   """
   Extracts the answer from the GSM8K dataset examples.

   Args:
       text (str): The dataset example text containing a question and answer.

   Returns:
       str or None: The extracted answer part after the '####' delimiter, or None if not found.

   Explanation:
        1. 检查文本是否包含用于分隔问题和答案的 `####` 分隔符。
        2. 如果找到，则在分隔符处分割文本并返回第二部分（即答案）。
        3. 答案会去除首尾的空白字符。
        4. 如果不存在分隔符，则返回 None。
   """
   if "####" not in text:
       return None
   return text.split("####")[1].strip()

在上述代码中：

- **`SYSTEM_PROMPT`：** 该字符串变量指示模型在 `<reasoning>` 标签中生成其思维链，并在 `<answer>` 标签中生成最终答案。使用这种一致的格式使得提取和评估答案变得更加容易。
- **`extract_answer_from_model_output`：** 该函数通过 `<answer>` 标签分割生成的文本，确保仅提取最后一次出现的标签中的内容。如果标签缺失或答案无效（例如，它是一个占位符 `"..."`），则该函数返回 `None`。
- **`extract_answer_from_dataset`：** 鉴于GSM8K数据集使用分隔符（`"####"`）来分隔答案，该函数通过在该分隔符上分割文本来提取预期的答案。

## 第三部分：数据集准备

在这一部分中，我们为训练准备GSM8K数据集。GSM8K是一个包含8.5K个高质量、语言多样的小学数学文字问题的数据集，由人类问题编写者创建。我们将使用该数据集中的示例来在强化学习（RL）范式中训练我们的模型：模型将生成多个问题解决方案样本，我们将这些解决方案与GSM8K示例中的真实数字进行比较，如果匹配，我们将为RL算法（GRPO）提供高奖励，从而更新模型的权重，以提高下次获得高奖励的概率。

我们首先从Hugging Face加载数据集，然后将每个示例**格式化为包含系统提示和用户提示**。我们还从数据集中提取预期的答案。这里定义了两个辅助函数：

1. **`prepare_dataset`：** 通过创建一个包含系统提示（带有格式化指令）和用户消息（问题）的提示来加载和准备GSM8K数据集。它还从数据集中提取答案。
2. **`build_prompt`：** 将消息字典列表连接为单个提示字符串。这确保了在训练和推理过程中提示构建的一致性。


In [3]:
def prepare_dataset(split="train"):
    """
    Load and prepare the GSM8K dataset for training with string prompts.

    Args:
        split (str): The dataset split to load ("train" or "test"). Defaults to "train".

    Returns:
       list: A list of formatted examples, each containing a prompt string and answer.

    Explanation:
        1. 从 Hugging Face 的 datasets hub 加载 GSM8K 数据集。
        2. 对于数据集中的每个示例：
        - 创建一个包含系统提示和问题的消息列表。
        - 使用 `build_prompt()` 将此列表转换为单个字符串提示。
        - 从数据集示例中提取答案。
        - 创建一个包含提示和答案的格式化示例字典。
        3. 返回准备好的格式化示例列表，用于模型训练或评估。
    """
    data = load_dataset('openai/gsm8k', 'main', cache_dir="./dataset", download_mode="reuse_dataset_if_exists")[split]
    formatted_data = []

    tmp = data[0]
    print("-----------打印示例数据----------")
    print("question: \n", tmp["question"])
    print("answer: \n", tmp["answer"])

    for example in data:
        # 将消息列表转换为单个字符串
        prompt_str = build_prompt([
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["question"]}
        ])
        formatted_example = {
            "prompt": prompt_str,  # Now a string rather than a list.
            "answer": extract_answer_from_dataset(example["answer"])
        }
        formatted_data.append(formatted_example)
    return formatted_data

def build_prompt(messages):
   """
   Build a single prompt string from a list of messages.

   Args:
       messages (list): A list of message dictionaries, each with 'role' and 'content' keys.

   Returns:
       str: A concatenated string of all message contents.

   Explanation:
        1. 获取一个以典型聊天格式组织的消息字典列表。
        2. 从每条消息中提取 `content` 字段并去除空白字符。
        3. 将所有内容字符串用换行符连接，生成一个单一的提示。
        4. 这种方法在从结构化消息转换为字符串的同时，保留了训练格式。
   """
   return "\n".join([msg["content"].strip() for msg in messages])

## 第四部分：评估函数

评估对于跟踪模型的进展至关重要。在这一部分中，我们定义了一些函数，使我们能够在一组示例上评估模型。评估函数执行以下任务：

- **对提示进行分词并生成响应：** 给定分词后的提示，生成模型的输出。
- **提取预测的答案：** 从生成的响应中提取答案。
- **将预测的答案与预期答案进行比较：** 通过精确匹配和数值等价性检查来完成这一比较。

两个辅助函数 `_extract_last_number` 和 `_extract_single_number` 用于从文本中提取数字。主要的评估函数 `evaluate_model` 使用这些辅助函数来确定预测的答案是否正确：


In [4]:
# 从文本中提取最后一个数字
def extract_last_number(text):
   """
   Extracts the last number appearing in the text.

   Args:
       text (str): The text to extract a number from.

   Returns:
       float or None: The last number in the text, or None if no number is found.

   Explanation:
       1. Removes dollar signs and percent symbols from the text.
       2. Uses regex to find a number that appears at the end of the text (possibly after whitespace).
       3. The pattern matches numbers that appear at the end of the string, with or without decimal points.
       4. Returns the found number as a float, or None if no match is found.
   """
   # 清理文本中的特殊符号：移除美元符号($)和百分号(%)
   text = text.replace('$', '').replace('%', '')
   # 正则表达式模式匹配行尾的数值（支持负数、小数、空格和等号前缀）
   pattern = r'(?:^|\s|=)\s*(-?\d*\.?\d+)\s*$'
   # 在文本中搜索匹配模式的内容
   match = re.search(pattern, text)
   # 返回提取的数值（浮点数），若未匹配则返回None
   return float(match.group(1)) if match else None

# 只有一个数字的情况下使用，其余情况返回None
def extract_single_number(text):
   """
   Extracts a single number from text if exactly one number is present.

   Args:
       text (str): The text to extract a number from.

   Returns:
       float or None: The single number in the text, or None if zero or multiple numbers are found.

   Explanation:
       1. Uses regex to find all numbers in the text (including negative numbers and decimals).
       2. If exactly one number is found, returns it as a float.
       3. If zero or multiple numbers are found, returns None.
   """
   numbers = re.findall(r'-?\d*\.?\d+', text)
   return float(numbers[0]) if len(numbers) == 1 else None

def evaluate_model(model, tokenizer, eval_examples, device):
   """
   Evaluates the model on a set of examples and prints detailed results.

   Args:
        model: 待评估的语言模型。
        tokenizer: 用于编码输入和解码输出的分词器。
        eval_examples (list): 评估示例列表，每个示例包含 "prompt" 和 "answer"。
        device: 运行评估的设备, CPU 或 GPU。


   Returns:
       float: The accuracy percentage (correct predictions / total examples * 100).

   Explanation:
        1. 将模型设置为评估模式。
        2. 对于评估集中的每个示例：
            - 对提示进行编码并使用模型生成响应。
            - 从生成的响应中提取预测的答案。
            - 使用多种方法将预测的答案与预期的答案进行比较：
                a. 精确字符串匹配
                b. 单数字提取和比较
                c. 最后一个数字提取和比较
            - 打印每个示例的详细信息。
        3. 计算并返回总体准确率。
        4. 将模型恢复为训练模式。

   """
   # 将模型设置为评估模式
   model.eval()
   correct = 0
   total = len(eval_examples)
   print("\n" + "="*50)
   print("EVALUATION ON", total, "EXAMPLES")
   print("="*50)

   for example in eval_examples:
       # Get the prompt and expected answer
       full_prompt = example["prompt"]
       expected = example["answer"]

       # Tokenize and generate response
       inputs = tokenizer.encode(full_prompt, return_tensors="pt").to(device)
       with torch.no_grad():
           outputs = model.generate(
               inputs,                                      # 输入张量
               max_new_tokens=512,                          # 最大生成 token 数量
               temperature=0.7,                             # 控制生成文本的随机性
               num_return_sequences=1,                      # 只生成一个输出序列
               pad_token_id=tokenizer.pad_token_id,         # 填充 token 的 ID
               eos_token_id=tokenizer.eos_token_id,         # 结束 token 的 ID
               forced_eos_token_id=tokenizer.eos_token_id,  # 强制在生成结束时添加结束 token
               early_stopping=False,                        # 不启用早停机制（生成固定长度）
           )
       # 将生成的 token 序列解码为可读文本
       response = tokenizer.decode(outputs[0], skip_special_tokens=True)

       try:
           # Extract answer and check correctness
           predicted = extract_answer_from_model_output(response)

           # Try different matching methods
           if predicted == expected:  # 完全精确匹配
               is_correct = True
           else:
               # 尝试提取单个数值进行匹配
               pred_num = extract_single_number(str(predicted)) # 从模型答案中提取数值
               exp_num = extract_single_number(str(expected))   # 从正确答案中提取数值
               if pred_num is not None and exp_num is not None and pred_num == exp_num:
                   is_correct = True
               else:
                   # 尝试提取最后一个数值进行匹配
                   pred_num = extract_last_number(str(predicted))   # 从模型答案中提取数值    
                   exp_num = extract_last_number(str(expected))     # 从正确答案中提取数值
                   is_correct = (pred_num is not None and exp_num is not None and
                               pred_num == exp_num)

           # 有一个对就算对
           if is_correct:
               correct += 1

           # Print evaluation details
        #    print("\nPrompt:")
        #    print(full_prompt)
        #    print("\nExpected Answer:")
        #    print(expected)
        #    print("\nExtracted Answer:")
        #    print(predicted)
        #    print("\nFull Generated Response:")
        #    print(response)
        #    print("\nCorrect:", "✓" if is_correct else "✗")
        #    print("-"*50)

       except Exception as e:
           print("\nFailed to parse model output for prompt:")
           print(full_prompt)
           print("Error:", e)
           print("-"*50)

   # Calculate and print final accuracy
   accuracy = (correct / total) * 100
   print(f"\nAccuracy: {accuracy:.2f}% ({correct}/{total})")
   print("="*50)

   # Return model to training mode
   model.train()
   return accuracy

在上述代码中：

- `_extract_last_number` 从文本字符串中提取最后一个数值，确保它被正确分隔且没有多余的符号。
- `_extract_single_number` 尝试从字符串中提取一个单一的数值，并且仅在找到一个数字时返回它。
- `evaluate_model`：  
  - 将模型设置为评估模式。
  - 遍历每个评估示例，构建提示，对其进行分词，并生成响应。
  - 提取预测的答案，并使用精确匹配和数值等价性检查（使用辅助函数）将其与预期答案进行比较。
  - 记录并打印每个示例的详细评估信息，并计算总体准确率。

## 第五部分：奖励函数

在强化学习中，奖励函数通过提供对模型输出的反馈来指导训练过程。在我们的pipeline中，我们定义了两个奖励函数：

1. **`correctness_reward`：**  
   该函数根据生成的答案是否正确来分配奖励。它将从模型输出中提取的答案与预期答案进行比较，使用精确字符串匹配和数值等价性检查。精确匹配会获得更高的奖励（2.0），而基于数值等价性的匹配会获得较小的奖励（1.5）。
   
2. **`format_reward`：**  
   该函数鼓励模型遵循所需的类似XML的输出格式。它为生成的文本中存在 `<reasoning>`、`</reasoning>`、`<answer>` 和 `</answer>` 标签提供少量的奖励。我们为这四个部分使用相对较小的值0.05，因为模型已经能够从前面的监督微调步骤中使用这些标签，所以我们给予这个小奖励，以确保它不会因为RL更新而忘记这样做。


In [5]:
# 准确性奖励(精准匹配2分, 答案匹配1.5分, 其余0分)
def correctness_reward(prompts, completions, answer, **kwargs):
    # 从模型输出中提取<answer>和</answer>之间的答案
    responses = [completion[0]['content'] for completion in completions]
    extracted = [extract_answer_from_model_output(r) for r in responses]
    rewards = []
    for r, a in zip(extracted, answer):
        # 完全匹配给2分
        if r == a:  # Exact match case
            rewards.append(2.0)
        # 不完全匹配给1.5分（数值答案正确）
        else:    
            # Try numeric equivalence
            r_num = extract_single_number(str(r))
            a_num = extract_single_number(str(a))
            if r_num is not None and a_num is not None and r_num == a_num:
                rewards.append(1.5)
            else:
                rewards.append(0.0)
   # Log completion lengths
    completion_lengths = [len(response.split()) for response in responses]
    return rewards

# 格式奖励(关于<reasoning> <reasoning> <answer> <answer>)
def format_reward(completions, **kwargs):
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    format_scores = []
    for response in responses:
        score = 0.0
        if "<reasoning>" in response: score += 0.2
        if "</reasoning>" in response: score += 0.2
        if "<answer>" in response: score += 0.2
        if "#### " in response: score += 0.2
        if "</answer>" in response: 
            score += 0.2
            # 冗余输出惩罚
            answer_end = response.find("</answer>") + len("</answer>")
            extra_text = response[answer_end:].strip()
            score = score * 0.5 if extra_text else score   # 如果"</answer>"后面还有别的输出，则格式分得分折半
        rewards.append(score)
        format_scores.append(score)
    return rewards

# 计算总得分
def combined_reward(prompts, completions, answer):
   """
   Combines correctness and format rewards.

   Args:
       prompts (list[str]): List of prompt texts
       completions (list[list[dict]]): List of completion dictionaries
       answer (list[str]): List of expected answers

   Returns:
       list[float]: Combined rewards for each prompt-completion pair

   Explanation:
       1. Calculates separate rewards for correctness and format compliance.
       2. Combines the rewards with the following weights:
          - Correctness score range: 0.0 to 2.0
          - Format score range: 0.0 to 1.0
          - Total possible range: 0.0 to 3.0
       3. Returns the combined reward for each example.
   """
   # Get individual rewards
   correctness_scores = correctness_reward(prompts=prompts, completions=completions, answer=answer)
   format_scores = format_reward(completions=completions)

   # Combine rewards - correctness is weighted more heavily
   combined_rewards = []
   for c_score, f_score in zip(correctness_scores, format_scores):
       # Correctness score range: 0.0 to 2.0
       # Format score range: 0.0 to 1.0
       # Total range: 0.0 to 2.8
       combined_rewards.append(c_score + f_score)

   return combined_rewards

## 第六部分：从零开始实现DataParallel GRPO

在本节中，我们从零开始实现GRPO算法的所有构建模块。该实现假设运行代码的机器至少具有2个GPU。我们使用PyTorch的 `DataParallel` API 将策略模型分布在GPU核心上，每个GPU核心上运行一个模型副本。批次数据在GPU核心之间进行分割。


In [6]:
def selective_log_softmax(logits, input_ids):
    """
    计算词汇表中特定 token 的对数概率。

    参数:
        logits (torch.Tensor): 模型输出的原始 logits（未归一化）。
        input_ids (torch.Tensor): 需要获取对数概率的目标 token ID。

    返回:
        torch.Tensor: 选中 token 的对数概率，形状与 `input_ids` 一致。

    实现步骤:
        1. 对 logits 应用 log_softmax，得到词汇表维度上的对数概率分布。
        2. 使用 `gather` 提取与 input_ids 对应的特定 token 的对数概率。
        3. 压缩多余的维度，使输出形状与 input_ids 匹配。
    """
    # 对 logits 应用 log_softmax（沿词汇维度归一化）
    log_probs = nn.functional.log_softmax(logits, dim=-1)
    
    # 从 log_probs 中提取 input_ids 对应的值，并压缩最后一个维度
    return log_probs.gather(dim=-1, index=input_ids.unsqueeze(-1)).squeeze(-1)



def compute_log_probs(model, input_ids, attention_mask, logits_to_keep):
    """
    计算输入序列末尾指定数量 token 的对数概率。

    参数:
        model: 语言模型（如 HuggingFace 的预训练模型）。
        input_ids (torch.Tensor): 输入序列的 token ID，形状为 [batch_size, seq_len]。
        attention_mask (torch.Tensor): 注意力掩码，标识有效 token 位置，形状与 input_ids 一致。
        logits_to_keep (int): 需要计算对数概率的末尾 token 数量（保留最后几个 token）。

    返回:
        torch.Tensor: 选中 token 的对数概率，形状为 [batch_size, logits_to_keep]。

    实现步骤:
        1. 获取模型输出的 logits，并移除最后一个位置的预测（因预测的是下一个 token）。
        2. 从 input_ids 中截取最后 `logits_to_keep` 个 token 作为目标。
        3. 从 logits 中截取对应的最后 `logits_to_keep` 个位置的预测结果。
        4. 调用 `selective_log_softmax` 计算目标 token 的对数概率。
    """
    # 获取模型输出 logits，并移除最后一个 token 的预测（形状变为 [batch_size, seq_len-1, vocab_size]）
    logits = model(input_ids=input_ids, attention_mask=attention_mask).logits[:, :-1, :]
    
    # 截取 input_ids 的最后 logits_to_keep 个 token（目标 token）
    input_ids = input_ids[:, -logits_to_keep:]
    
    # 截取 logits 的最后 logits_to_keep 个位置（对应目标 token 的预测）
    logits = logits[:, -logits_to_keep:, :]
    
    # 计算目标 token 的对数概率
    return selective_log_softmax(logits, input_ids)



def create_completion_mask(completion_ids, eos_token_id):
    """
    生成一个掩码，用于屏蔽生成序列中首个 EOS token 之后的所有 token。

    参数:
        completion_ids (torch.Tensor): 生成序列的 token ID，形状为 [batch_size, seq_len]。
        eos_token_id (int): 结束符（EOS）的 token ID。

    返回:
        torch.Tensor: 二进制掩码，形状为 [batch_size, seq_len]，有效 token 为 1，EOS 之后为 0。

    实现步骤:
        1. 标记 EOS 出现的位置。
        2. 初始化默认 EOS 索引为序列长度（处理无 EOS 的情况）。
        3. 确定哪些序列中存在 EOS。
        4. 对有 EOS 的序列，更新首个 EOS 的位置索引。
        5. 生成位置索引矩阵，并与 EOS 索引比较生成掩码。
    """
    # 标记 EOS token 的位置（布尔张量）
    is_eos = completion_ids == eos_token_id

    # 初始化默认 EOS 索引为序列长度（若无 EOS，则保留全部 token）
    eos_idx = torch.full(
        (is_eos.size(0),), 
        is_eos.size(1),  # 默认值为序列长度 seq_len
        dtype=torch.long, 
        device=completion_ids.device
    )

    # 检查每个序列是否存在至少一个 EOS
    mask_exists = is_eos.any(dim=1)

    # 对有 EOS 的序列，获取首个 EOS 的位置索引
    # （利用 argmax 返回第一个 True 的位置）
    eos_idx[mask_exists] = is_eos.int().argmax(dim=1)[mask_exists]

    # 生成位置索引矩阵 [batch_size, seq_len]
    # 例如：若 seq_len=3，则每行为 [0, 1, 2]
    sequence_indices = torch.arange(
        is_eos.size(1), 
        device=completion_ids.device
    ).expand(is_eos.size(0), -1)

    # 生成掩码：位置索引 <= 首个 EOS 索引的位置设为 1
    return (sequence_indices <= eos_idx.unsqueeze(1)).int()



def generate_completions(model, tokenizer, prompts, num_generations=4, max_completion_length=32):
    """
    为每个提示(prompt)生成多个补全(completion)。

    Args:
        model: 语言模型对象
        tokenizer: 用于文本编码/解码的分词器
        prompts (list): 文本提示列表
        num_generations (int): 每个提示生成的补全数量
        max_completion_length (int): 生成的最大token数量

    Returns:
        tuple: 包含四个元素的元组，格式为 (prompt_ids, prompt_mask, completion_ids, completion_mask)

    Explanation:
        1. 编码输入提示(prompts)并将其移动到计算设备
        2. 每个提示重复多次以实现批量生成
        3. 使用模型生成补全内容
        4. 从输出中分离生成的补全部分
        5. 创建补全的注意力掩码
    """
    # 选择计算设备（优先使用GPU）
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # 编码输入提示(prompts)
    inputs = tokenizer(
        prompts, 
        return_tensors="pt",  # 返回PyTorch tensor格式
        padding=True,         # 启用填充
        padding_side="left"   # 左侧填充
    )
    
    # 将输入数据移动到计算设备
    prompt_ids = inputs["input_ids"].to(device)
    prompt_mask = inputs["attention_mask"].to(device)
    
    # 打印调试信息（输入批量大小和设备）
    # print(f"Input batch size: {prompt_ids.size(0)}, Device before model: {prompt_ids.device}")
    
    # 记录提示文本的原始长度（用于后续截取生成的补全）
    prompt_length = prompt_ids.size(1)
    
    # 沿批次维度(batch dimension)重复每个提示
    # 示例：num_generations=4时将批次扩大4倍（每个原始样本生成4个补全）
    prompt_ids = prompt_ids.repeat_interleave(num_generations, dim=0)
    prompt_mask = prompt_mask.repeat_interleave(num_generations, dim=0)
    
    # 使用模型生成文本
    outputs = model.generate(
        input_ids=prompt_ids,
        attention_mask=prompt_mask,
        max_new_tokens=max_completion_length,  # 生成的最大新token数
        do_sample=True,         # 启用随机采样
        temperature=1.0,        # 采样温度参数
        pad_token_id=tokenizer.pad_token_id,   # 填充token ID
        eos_token_id=tokenizer.eos_token_id,   # 结束token ID
        early_stopping=False    # 禁用提前停止（生成指定长度）
    )
    
    # 打印生成后的调试信息
    print(f"Output batch size: {outputs.size(0)}, Device after model: {outputs.device}")
    
    # 提取生成的补全部分（截取掉原始的提示内容）
    completion_ids = outputs[:, prompt_length:]
    
    # 创建补全的注意力掩码（用于后续处理）
    completion_mask = create_completion_mask(completion_ids, tokenizer.eos_token_id)
    
    return prompt_ids, prompt_mask, completion_ids, completion_mask



def generate_rollout_data(model, ref_model, tokenizer, batch_samples, num_generations, max_completion_length):
    """
    生成用于GRPO训练的交互数据，包括生成文本及其对数概率。

    Args:
        model: 当前训练的强化学习策略模型
        ref_model: 参考模型（用于KL散度计算的基准模型）
        tokenizer: 文本编码/解码器
        batch_samples (list): 训练样本组成的批次（每个样本包含prompt和answer）
        num_generations (int): 每个样本生成的补全数量
        max_completion_length (int): 生成文本的最大token长度

    Returns:
        dict: 包含GRPO训练所需完整数据的字典

    Explanation:
        1. 解析输入批次中的提示和标准答案
        2. 使用当前模型生成多条补全文本
        3. 拼接提示与生成文本得到完整序列
        4. 计算策略模型和参考模型的生成文本对数概率
        5. 格式化生成文本以计算奖励
        6. 对齐输入数据维度（根据生成数量扩展样本）
    """
    # 检测计算设备（优先使用GPU）
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # 从训练样本中提取提示和答案
    # 支持字典和元组两种数据格式
    # 示例：
    # 'prompt': 'Respond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'
    # 'answer': '72'
    prompts = [sample["prompt"] if isinstance(sample, dict) else sample[0] for sample in batch_samples]
    answers = [sample["answer"] if isinstance(sample, dict) else sample[1] for sample in batch_samples]

    # 禁用梯度计算（生成阶段）
    with torch.no_grad():
        # 生成多个补全文本 (shape: (batch_size * num_generations, ...))"
        prompt_ids, prompt_mask, completion_ids, completion_mask = generate_completions(
            model, tokenizer, prompts, num_generations, max_completion_length
        )
        
        # 拼接提示和补全作为完整输入序列
        # 维度: (batch_size*num_generations, prompt_len+completion_len)
        input_ids = torch.cat([prompt_ids, completion_ids], dim=1)             
        attention_mask = torch.cat([prompt_mask, completion_mask], dim=1)     # 相同的维度结构
        
        # 记录需要保留的logits数量（补全文本的token长度）
        logits_to_keep = completion_ids.size(1)
        
        # 计算模型对生成内容的对数概率
        old_log_probs = compute_log_probs(model, input_ids, attention_mask, logits_to_keep)       # 当前策略模型
        ref_log_probs = compute_log_probs(ref_model, input_ids, attention_mask, logits_to_keep)   # 参考模型

    # 解码生成文本用于后续奖励计算
    formatted_completions = [[
        {'content': tokenizer.decode(ids, skip_special_tokens=True)}  # 解码并跳过特殊token
    ] for ids in completion_ids]  # 每个生成样本包装成字典列表形式

    # 扩展原始样本以匹配生成的补全数量
    repeated_prompts = [p for p in prompts for _ in range(num_generations)]    # 格式示例：[prompt1, prompt1, prompt1..., prompt2, prompt2...]
    repeated_answers = [a for a in answers for _ in range(num_generations)]   # 同理扩展答案

    # 封装所有训练数据
    return {
        "input_ids": input_ids,                    # 完整输入序列的token ID
        "attention_mask": attention_mask,          # 对应的注意力掩码
        "completion_mask": completion_mask,        # 仅补全部分的掩码
        "old_log_probs": old_log_probs,            # 策略模型的对数概率
        "ref_log_probs": ref_log_probs,            # 参考模型的对数概率（用于KL约束）
        "formatted_completions": formatted_completions,  # 可用于奖励模型的可读文本
        "repeated_prompts": repeated_prompts,      # 扩展后的提示文本（与生成样本对齐）
        "repeated_answers": repeated_answers,      # 扩展后的答案（与生成样本对齐）
        "logits_to_keep": logits_to_keep,          # 补全文本的token长度
        "batch_size": len(prompts),                # 原始批次数目
        "num_generations": num_generations         # 每个样本生成数
    }



def grpo_loss(model, ref_model, rollout_data, tokenizer, reward_function, beta=0.01, epsilon=0.2):
    """
    计算GRPO（Generalized Reinforcement Policy Optimization）损失函数。

    Args:
        model: 当前训练的策略模型
        ref_model: 参考模型（用于KL散度计算）
        rollout_data (dict): 由generate_rollout_data生成的交互数据
        tokenizer: 文本编码/解码器
        reward_function: 奖励计算函数
        beta (float): KL散度惩罚系数（默认0.01）
        epsilon (float): PPO裁剪范围参数（默认0.2）

    Returns:
        torch.Tensor: 需要最小化的GRPO损失值
        float: 当前批次的平均奖励值

    Explanation:
        实现GRPO算法的核心损失计算，包含：
        - 策略梯度优化（PPO裁剪机制）
        - KL散度约束（防止策略过激偏离参考模型）
        - 奖励标准化处理
    """
    # 自动检测计算设备
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # 从交互数据中提取关键元素
    input_ids = rollout_data["input_ids"]  # 完整输入序列的token ID (batch_size*num_gen, seq_len)
    attention_mask = rollout_data["attention_mask"]  # 注意力掩码
    completion_mask = rollout_data["completion_mask"]  # 补全部分的掩码（仅生成部分为1）
    logits_to_keep = rollout_data["logits_to_keep"]  # 补全文本的token长度
    old_log_probs = rollout_data["old_log_probs"]  # 旧策略的对数概率
    ref_log_probs = rollout_data["ref_log_probs"]  # 参考模型的对数概率

    # 1. 计算当前策略模型的对数概率, 梯度可追踪
    token_log_probs = compute_log_probs(model, input_ids, attention_mask, logits_to_keep)
    
    # 2. 新旧策略概率比率（指数运算将log概率转换为概率比值）
    ratio = torch.exp(token_log_probs - old_log_probs)  # 形状: (batch_size*num_gen, seq_len)

    # 3. 计算奖励值 
    rewards = torch.tensor(
        reward_function(
            prompts=rollout_data["repeated_prompts"],  # 扩展后的提示列表
            completions=rollout_data["formatted_completions"],  # 格式化的生成文本
            answer=rollout_data["repeated_answers"]  # 扩展后的标准答案
        ),    # 把模型的输出输入到奖励计算函数，即combined_reward
        dtype=torch.float32,
        device=device
    )  

    # 4. 维度调整与奖励标准化 （广义优势函数）
    batch_size = rollout_data["batch_size"]  # 原始批次数
    num_generations = rollout_data["num_generations"]  # 每个样本生成数
    
    # 将奖励重塑为矩阵形式： (batch_size, num_generations)
    rewards = rewards.view(batch_size, num_generations)
    
    # 计算并打印平均奖励（用于监控训练进度）
    avg_reward = rewards.mean().item()
    print("Average Reward:", avg_reward)
    
    # 计算每个提示对应的平均奖励和标准差（用于标准化）
    mean_rewards = rewards.mean(dim=1).repeat_interleave(num_generations)  # (batch_size*num_gen,)
    std_rewards = rewards.std(dim=1).repeat_interleave(num_generations)    
    
    # 优势函数计算（标准化处理）
    advantages = ((rewards.view(-1) - mean_rewards) / (std_rewards + 1e-4)).unsqueeze(1)  

    # 5. PPO裁剪目标计算 
    surr1 = ratio * advantages          # 未裁剪的原始目标
    surr2 = torch.clamp(ratio, 1 - epsilon, 1 + epsilon) * advantages  # 裁剪后的保守目标
    surrogate_loss = torch.min(surr1, surr2)  # 取两者较小值 → 形状: (batch_size*num_gen, seq_len)

    # 6. KL散度惩罚项计算 
    # 使用近似公式：KL(p||q) ≈ exp(log_p - log_q) - (log_p - log_q) - 1
    kl = torch.exp(ref_log_probs - token_log_probs) - (ref_log_probs - token_log_probs) - 1

    # 7. 综合损失计算 
    per_token_loss = surrogate_loss - beta * kl  
    
    # 应用补全掩码（仅计算生成部分的损失）并平均
    masked_loss = per_token_loss * completion_mask  # 形状保持不变
    loss = -((masked_loss.sum(dim=1) / completion_mask.sum(dim=1)).mean())  # 负号因为要最大化奖励
    
    return loss, avg_reward

# 1. 导入PEFT库进行LoRA配置
from peft import LoraConfig, get_peft_model
def train_with_grpo(model, tokenizer, train_data, num_iterations=1, num_steps=500, batch_size=4,
                              num_generations=4, max_completion_length=128, beta=0.1,
                              learning_rate=5e-6, mu=3, epsilon=0.2, reward_function=None, device_ids=None):
    """
    使用GRPO算法进行迭代式强化学习训练。

    Args:
        model: 待训练的语言模型
        tokenizer: 文本编码/解码器
        train_data (list): 训练数据集
        num_iterations (int): 外层迭代次数（参考模型更新次数）
        num_steps (int): 每个迭代的批次更新次数
        batch_size (int): 每批次的提示样本数
        num_generations (int): 每个提示生成的补全数量
        max_completion_length (int): 生成文本的最大token长度
        beta (float): KL散度惩罚系数
        learning_rate (float): 优化器学习率
        mu (int): 每个批次的策略更新次数
        epsilon (float): PPO裁剪参数
        reward_function: 奖励计算函数
        device_ids (list): 多GPU设备ID列表

    Returns:
        训练完成的模型

    Explanation:
        实现GRPO算法的完整训练流程，包含：
        - 多GPU并行训练支持
        - 参考模型的定期更新
        - 多轮策略优化步骤
        - 训练指标监控
    """
    # 断言确保多GPU配置有效
    assert device_ids is not None and len(device_ids) > 1, "需要至少2个GPU核心运行本代码!"  

    # 配置tensorboard 
    writer = SummaryWriter(log_dir="runs/dlc_1.5B_experiment")  

    # 自动检测计算设备
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # 多GPU并行处理设置 
    # 2. 配置LoRA参数
    # lora_config = LoraConfig(
    #     r=8,  # 秩
    #     lora_alpha=32,
    #     target_modules=["q_proj", "v_proj"],  # 根据模型结构调整
    #     lora_dropout=0.1,
    #     bias="none",
    #     task_type="CAUSAL_LM"
    # )
    lora_config = LoraConfig(
                                r=8,
                                lora_alpha=16,
                                lora_dropout=0.2,
                                target_modules=["q_proj", "k_proj", "v_proj"],  # 增加k_proj
                                bias="all",
                                task_type="CAUSAL_LM",
                                modules_to_save=["lm_head"]  # 解锁输出层
                            )

    # 3. 获取新训练模型
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters() 
    model = nn.DataParallel(model, device_ids=device_ids)  # 包装为数据并行模型
    print(f"模型已部署到多GPU: {device_ids}")
    
    # 外层循环：参考模型更新迭代 
    for iteration in range(num_iterations):
        print(f"\n当前迭代 {iteration+1}/{num_iterations}")

        # 创建参考模型（深拷贝当前模型）
        ref_model = copy.deepcopy(model.module)  # 注意通过.module访问原始模型
        ref_model.eval()  # 设置为评估模式
        for param in ref_model.parameters():
            param.requires_grad = False  # 冻结参考模型参数
        print("参考模型创建完成.")

        # 优化器重新初始化 -----------------------------------------------------
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        model.train()  # 确保策略模型处于训练模式

        # 内层循环：训练批次处理 ------------------------------------------------
        for step in range(num_steps):
            # 1. 采样训练批次
            batch_samples = random.sample(train_data, batch_size)
            
            # 2. 生成交互数据（禁用梯度计算）
            with torch.no_grad():
                rollout_data = generate_rollout_data(
                    model.module,  # 使用原始模型（非DataParallel包装）
                    ref_model,
                    tokenizer,
                    batch_samples,
                    num_generations,
                    max_completion_length
                )
            
            # 3. 多轮策略优化（mu次参数更新）
            for grpo_iter in range(mu):
                # 计算GRPO损失
                loss, avg_reward = grpo_loss(
                    model.module,
                    ref_model,
                    rollout_data,
                    tokenizer,
                    reward_function,
                    beta=beta,
                    epsilon=epsilon
                )
                
                # 梯度更新步骤
                optimizer.zero_grad()  # 清空梯度
                loss.backward()  # 反向传播
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)  # 梯度裁剪
                optimizer.step()  # 参数更新
                
                # wandb.log({
                #     "loss": loss.item(),
                #     "average_reward": avg_reward,
                #     "iteration": iteration + 1,
                #     "step": step + 1,
                #     "grpo_iter": grpo_iter + 1
                # })
                # 替换 wandb.log 的代码为：
                global_step = iteration * num_steps + step  # 需要计算全局步数
                writer.add_scalar('Loss/train', loss.item(), global_step) 
                writer.add_scalar('Reward/train', avg_reward, global_step)

                
                # 打印训练进度
                print(f"迭代 [{iteration+1}/{num_iterations}], 步骤 [{step+1}/{num_steps}], "
                      f"GRPO更新 [{grpo_iter+1}/{mu}], 损失: {loss.item():.4f}")
                print()
                
                # GPU监控（可选）
                # for i in range(torch.cuda.device_count()):
                #    print(f"GPU {i} 内存使用: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MiB, "
                #          f"利用率: {torch.cuda.utilization(i)}%")
    
    # 训练完成关闭 writer
    writer.close()

    # 返回原始模型（解除DataParallel包装）
    return model.module


## 第七部分：训练设置与执行

在本节中，我们将所有组件整合在一起以设置并运行训练。我们首先加载预训练模型和分词器，准备评估数据，然后使用我们上面从零开始实现的 `train_with_grpo` 进行强化学习（RL）微调。

关键步骤包括：

- **模型和分词器初始化：**  
  加载模型 `"Qwen/Qwen2.5-1.5B-Instruct"` 并应用优化设置（使用 `torch.bfloat16` 和 FlashAttention2）。同时加载分词器，并将其填充标记设置为序列结束标记。使用 `torch.bfloat16` 加载模型会将其参数从每个数字32位转换为16位，从而将模型的内存使用量减半，并可以在现代GPU上加快训练速度。
  
- **初始评估：**  
  在微调之前，对模型进行一些示例的评估，以建立基线性能。
    
- **强化学习微调（RL）：**  
  使用我们实现的 `train_with_grpo` 函数进行GRPO训练，配置适当的训练参数和奖励函数。然后在剩余的训练数据上进行RL训练。
  
- **最终评估和模型保存：**  
  RL微调完成后，再次评估模型，并保存最终的模型。

在以下代码中：
  
- 确定设备（优先使用GPU，否则使用CPU）。
- 加载预训练的Qwen2.5-1.5B-Instruct模型和分词器。分词器的填充标记设置为eos_token。
- 保留数据集的一小部分用于评估，以提供基线。
- 通过启用梯度检查点和禁用KV缓存来优化模型的内存效率。
- **步骤1：** 在微调之前评估模型，以建立基线准确率。
- **步骤2：** 使用 `train_with_grpo` 函数进行强化学习微调，结合我们定义的奖励函数（`format_reward` 和 `correctness_reward`，组合为 `combined_reward`）。模型使用多GPU进行训练。
- **步骤3：** 将最终微调后的模型和分词器保存到磁盘。

我们为GRPO训练管道使用了以下超参数：

### **训练配置**

这些参数配置了使用GRPO算法进行强化学习微调的运行。我们设置如下：

- **num_iterations=1**  
  外部迭代次数，每次迭代从当前策略模型创建一个新的参考模型。一次迭代是对整个数据集的一次遍历。

- **num_steps=500**  
  训练循环最多执行500步，每步处理一批示例。

- **batch_size=7**  
  每步处理7个示例，在8个GPU的情况下，每个GPU处理1个示例。`DataParallel` 使用一个GPU（0）作为主GPU，用于聚合梯度和收集输出。

- **num_generations=14**  
  对于训练数据中的每个提示，训练器将生成14个不同的完成结果。这些多个生成结果用于计算相对优势（或奖励信号），以指导RL更新。如果GPU的VRAM较小，可以减少此数值。

- **max_completion_length=400**  
  生成完成结果（序列的“响应”部分）时，生成的长度限制为400个标记。这限制了RL阶段模型生成的输出长度。如果GPU的VRAM较小，可以减少此数值。

- **beta=0.04**  
  GRPO损失函数中KL散度惩罚的系数。这控制了模型与参考模型的偏离程度。

- **learning_rate=5e-6**  
  RL微调的学习率。使用相对较低的学习率以确保策略更新的稳定性。

- **mu=1**  
  每批滚动数据执行的策略更新次数。在我们的情况下，每批只执行一次更新。

- **epsilon=0.1**  
  GRPO中PPO组件的裁剪参数。这防止策略在单次更新中发生过大变化。

在微调前后对模型进行评估，以衡量准确率的提升。最后，将微调后的模型保存到 "grpo_finetuned_model" 目录中。


In [7]:
all_data = prepare_dataset("train")
random.shuffle(all_data)
size_of_eval_data = 30 # change to a smaller value to save time or to a larger number for a more reliable estimate
eval_data = all_data[:size_of_eval_data]
train_data = all_data[size_of_eval_data:]
print()
print(f"Number of training examples: {len(train_data)}")

Using the latest cached version of the dataset since openai/gsm8k couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'main' at dataset/openai___gsm8k/main/0.0.0/e53f048856ff4f594e959d75785d2c2d37b678ee (last modified on Wed Mar  5 02:24:00 2025).


-----------打印示例数据----------
question: 
 Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
answer: 
 Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72

Number of training examples: 7443


In [8]:
def optimize_model_memory(model):
    """
    Optimizes the model to use less memory during training.

    Args:
        model: The language model to optimize.

    Returns:
        The optimized model.

    Explanation:
        1. Sets the model to training mode.
        2. Disables KV caching to save memory.
        3. Enables gradient checkpointing to trade computation for memory.
        4. Ensures that input embeddings require gradients:
           - Either uses the built-in method if available.
           - Or adds a forward hook to the input embeddings layer.
        5. Returns the optimized model ready for memory-efficient training.
    """
    model.train()
    model.config.use_cache = False

    # First ensure inputs will require gradients
    if hasattr(model, "enable_input_require_grads"):
        model.enable_input_require_grads()
    else:
        def make_inputs_require_grad(module, input, output):
            output.requires_grad_(True)
        model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)

    # Then enable gradient checkpointing
    model.gradient_checkpointing_enable()

    return model

# Main execution
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using primary device: {device}")

# 获取及下载模型
# modelscope download --model Qwen/Qwen2.5-7B-Instruct --local_dir /home/yeqi3/cyr/code/Basic-LLM-Learning/Code/LLM05/models/Qwen2.5-7B-Instruct
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
output_dir = "math_solver_model"

print("Downloading model...")
# 这里用的是将模型下载到本地之后再指定模型路径进行读取
base_model_path = r"./models/Qwen2.5-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=base_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_cache=False
)
print("Model downloaded")

# 获取及下载分词器
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=base_model_path,
    padding_side="left",
    use_cache=False
)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id

# 获取显卡数量
num_gpus = torch.cuda.device_count()
print(f"Detected {num_gpus} GPUs")
device_ids = list(range(num_gpus)) if num_gpus > 1 else None

model = optimize_model_memory(model)

print("\nStarting RL fine-tuning using GRPO...")
training_config = {
    'num_iterations': 1,
    'num_steps': 500,
    'batch_size': 1,                # reduce if you have fewer GPUs
    'num_generations': 14,          # reduce if you have GPUs with less VRAM
    'max_completion_length': 400,   # reduce if you have GPUs with less VRAM
    'beta': 0.04,
    'learning_rate': 5e-6,
    'mu': 1,
    'epsilon': 0.1
}

print("Train Config:")
print(training_config)

# Initialize Weights & Biases
print("Weights & Biases initialized.")

model = train_with_grpo(
    model=model,
    tokenizer=tokenizer,
    train_data=train_data,
    reward_function=combined_reward,
    device_ids=device_ids,
    **training_config
)

print("Training completed and wandb run finished.")

print("\nFinal model evaluation after GRPO RL fine-tuning:")
post_grpo_accuracy = evaluate_model(model, tokenizer, eval_data, device)
print(f"Post-GRPO Accuracy: {post_grpo_accuracy:.2f}%")

print("\nSaving GRPO fine-tuned model...")
model.save_pretrained("dlc_1.5B_model")
tokenizer.save_pretrained("dlc_1.5B_model")

Using primary device: cuda:0
Model downloaded
Detected 4 GPUs

Starting RL fine-tuning using GRPO...
Train Config:
{'num_iterations': 1, 'num_steps': 500, 'batch_size': 1, 'num_generations': 14, 'max_completion_length': 400, 'beta': 0.04, 'learning_rate': 5e-06, 'mu': 1, 'epsilon': 0.1}
Weights & Biases initialized.
trainable params: 234,921,984 || all params: 1,778,578,944 || trainable%: 13.2084
模型已部署到多GPU: [0, 1, 2, 3]

当前迭代 1/1
参考模型创建完成.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Output batch size: 14, Device after model: cuda:0
Average Reward: 0.22142858803272247
迭代 [1/1], 步骤 [1/500], GRPO更新 [1/1], 损失: 0.0000
Output batch size: 14, Device after model: cuda:0
Average Reward: 0.2071428894996643
迭代 [1/1], 步骤 [2/500], GRPO更新 [1/1], 损失: -0.0001
Output batch size: 14, Device after model: cuda:0
Average Reward: 0.6357143521308899
迭代 [1/1], 步骤 [3/500], GRPO更新 [1/1], 损失: 0.0005
Output batch size: 14, Device after model: cuda:0
Average Reward: 0.29285717010498047
迭代 [1/1], 步骤 [4/500], GRPO更新 [1/1], 损失: 0.0012
Output batch size: 14, Device after model: cuda:0
Average Reward: 0.22857144474983215
迭代 [1/1], 步骤 [5/500], GRPO更新 [1/1], 损失: 0.0003
Output batch size: 14, Device after model: cuda:0
Average Reward: 0.3857143521308899
迭代 [1/1], 步骤 [6/500], GRPO更新 [1/1], 损失: 0.0007
Output batch size: 14, Device after model: cuda:0
Average Reward: 0.02857143059372902
迭代 [1/1], 步骤 [7/500], GRPO更新 [1/1], 损失: 0.0004
Output batch size: 14, Device after model: cuda:0
Average Reward: 0.285

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


迭代 [1/1], 步骤 [500/500], GRPO更新 [1/1], 损失: 0.0005
Training completed and wandb run finished.

Final model evaluation after GRPO RL fine-tuning:

EVALUATION ON 30 EXAMPLES

Accuracy: 23.33% (7/30)
Post-GRPO Accuracy: 23.33%

Saving GRPO fine-tuned model...


('dlc_1.5B_model/tokenizer_config.json',
 'dlc_1.5B_model/special_tokens_map.json',
 'dlc_1.5B_model/vocab.json',
 'dlc_1.5B_model/merges.txt',
 'dlc_1.5B_model/added_tokens.json',
 'dlc_1.5B_model/tokenizer.json')

# 第八部分：对比模型效果

使用原始模型和GRPO训练后的模型进行训练，统一调用的是本代码中实现的`evaluate_model`函数。

In [9]:
# 加载原始模型（未优化的原始模型）
print("\nLoading original model for evaluation...")
original_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=base_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_cache=False  # 评估时启用缓存加速生成
)
original_model.eval()  # 设置为评估模式

# 加载微调后的模型
print("\nLoading fine-tuned model for evaluation...")
finetuned_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="dlc_1.5B_model",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_cache=False  # 评估时启用缓存加速生成
)
finetuned_model.eval()  # 设置为评估模式

# 确保分词器设置一致
original_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=base_model_path,
    padding_side="left"
)
original_tokenizer.pad_token = original_tokenizer.eos_token

# 评估原始模型
print("\nEvaluating original model...")
with torch.no_grad():
    original_accuracy = evaluate_model(
        model=original_model,
        tokenizer=original_tokenizer,
        eval_examples=eval_data,
        device=device
    )

# 评估微调后的模型
print("\nEvaluating fine-tuned model...")
with torch.no_grad():
    finetuned_accuracy = evaluate_model(
        model=finetuned_model,
        tokenizer=tokenizer,  # 使用训练时保存的分词器
        eval_examples=eval_data,
        device=device
    )

print(f"Original Model Accuracy: {original_accuracy:.2f}%")
print(f"Fine-tuned Model Accuracy: {finetuned_accuracy:.2f}%")


Loading original model for evaluation...

Loading fine-tuned model for evaluation...

Evaluating original model...

EVALUATION ON 30 EXAMPLES

Accuracy: 3.33% (1/30)

Evaluating fine-tuned model...

EVALUATION ON 30 EXAMPLES

Accuracy: 60.00% (18/30)
Original Model Accuracy: 3.33%
Fine-tuned Model Accuracy: 60.00%


正如你所看到的，模型没有学会生成序列结束（`EOS`）标记，因此输出序列在 `</answer>` 标签之后仍然继续。这是预期的行为，因为我们使用的奖励函数没有为停止生成提供特殊的奖励。我们也没有进行监督微调步骤（如我们[在这里](https://github.com/aburkov/theLMbook/blob/main/GRPO_Qwen_0_5_Instruct.ipynb)所做的那样），在那里我们可以教会模型在 `</answer>` 之后生成 `EOS`。

这就是本教程的结束。此时，你应该对GRPO算法的工作原理以及构建用于微调语言模型以完成数学、编码和逻辑任务的RL管道所需的组件有了清晰的理解。


<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        This was an extension notebook for <a href="https://www.thelmbook.com" target="_blank" rel="noopener">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a href="https://github.com/aburkov/theLMbook" target="_blank" rel="noopener">https://github.com/aburkov/theLMbook</a>
                        <br><br>
                        <a href="https://www.thelmbook.com" target="_blank" rel="noopener">Read the book</a> to learn about language modeling and train yours from scratch.
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>